In [1]:
#first it decodes the message and writes it as a text file:
import whisper

model = whisper.load_model("tiny")  # Options: tiny, base, small, medium, large

result = model.transcribe("command_test.mp3")
print(result["text"])

100%|█████████████████████████████████████| 72.1M/72.1M [00:04<00:00, 16.2MiB/s]
C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 test test test I'm gonna be talking like this let's see how it handles when it's super loud and then also when I'm really close and it's super loud or when I'm pretty far away and it's kind of quiet let's see if it can handle a little bit of this a little bit of that some flaps and I think birthday party power off run function meeting at 5 p.m. in new Plaza Square you need a doctor's appointment pretty soon make sure you get that going don't forget to visit the birthday party at 7 p.m.


first lets see if it can do this in real time. 
to do this, since the model is trained on 30 second intervals we will need to break the audio up into 30 second chunks.
then while its transcribing one section, it will be recording the next section, that way its going to be at most 30 seconds behind.
it will save these 30 second chunks to a file, labeled with date and time and then it will search through the file for any new information


In [9]:
#real time transcription to a file
import threading
import os
import numpy as np
import whisper
import sounddevice as sd
from scipy.io.wavfile import write
from pydub import AudioSegment
import time
from datetime import datetime

def record_audio_to_mp3(filename="output.mp3", duration=30, samplerate=16000):
    
    print(f"Recording audio for {duration} seconds...")

    # Record audio to a NumPy array
    recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()  # Wait until recording is finished

    # Save the recorded audio to a temporary WAV file
    temp_wav_filename = "temp_recording.wav"
    write(temp_wav_filename, samplerate, recording)
    print(f"Recorded audio saved as {temp_wav_filename}")

    # Convert the WAV file to MP3 using pydub
    audio = AudioSegment.from_wav(temp_wav_filename)
    audio.export(filename, format="mp3")
    print(f"Converted to MP3 and saved as {filename}")

    # Clean up the temporary WAV file (optional)
    import os
    os.remove(temp_wav_filename)
    print(f"Temporary WAV file '{temp_wav_filename}' removed.")


model = whisper.load_model("tiny")  # Options: tiny, base, small, medium, large


print(result["text"])
def record():
    #record for 30 seconds
    current_datetime = datetime.now()
    date_time = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
    date_time = date_time.replace(":", "-")  # Replace colons with dashes for filename compatibility
    record_audio_to_mp3(filename= "recording " + date_time + ".mp3", duration=10) 
    #transcribe
    result = model.transcribe("recording " + date_time + ".mp3") 
    #write to file
    with open("transcription.txt", "a") as f:
        f.write(result["text"] + "\n")
    #delete the file
    os.remove("recording " + date_time + ".mp3")
    print("Transcription complete and recording deleted.")
    pass

my_thread = threading.Thread(target=record, args=()) #set up the thread with the target function and its arguments
my_thread.start() #calls the start method to begin execution of the thread
my_thread.join() #waits for the thread to finish before continuing execution of the main program

 test test test I'm gonna be talking like this let's see how it handles when it's super loud and then also when I'm really close and it's super loud or when I'm pretty far away and it's kind of quiet let's see if it can handle a little bit of this a little bit of that some flaps and I think birthday party power off run function meeting at 5 p.m. in new Plaza Square you need a doctor's appointment pretty soon make sure you get that going don't forget to visit the birthday party at 7 p.m.
Recording audio for 10 seconds...
Recorded audio saved as temp_recording.wav
Converted to MP3 and saved as recording 2025-07-17 17-43-15.mp3
Temporary WAV file 'temp_recording.wav' removed.
Transcription complete and recording deleted.


In [ ]:
#code words and new information